In [148]:
import os
from langchain_openai import ChatOpenAI
from crewai import LLM

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

gpt_4o_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.1,
)

In [149]:
from pydantic import BaseModel
from typing import List

class filePath(BaseModel):
    filePath: List[str]

class associateFilePath(BaseModel):
    mainFile: str
    relatedFiles: List[str]
    imageFiles: List[str]

In [150]:
from crewai import Agent
from crewai_tools import FileReadTool, DirectoryReadTool


class Agents:
    def markdownPathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the markdown files inside {file_path} path. Should never modify the path of the file.",
            backstory="You are very good at finding markdown files.",
            allow_delegation=False,
            verbose=True,
            llm=gpt_4o_llm,
            tools=[
                DirectoryReadTool(),
            ],
        )

    def imgPathSearcher(self):
        return Agent(
            role="pathSearcher",
            goal="Finds the img files inside {img_path} path",
            backstory="You are fluent in Korean, and you are very good at finding image files.",
            allow_delegation=False,
            verbose=True,
            llm=gpt_4o_llm,
            tools=[
                DirectoryReadTool(),
            ],
        )

    def mainFilesearcher(self):
        return Agent(
            role="mainFilesearcher",
            goal="Print out ONLY one document that can answer {question}. To use the Tool, The parameter MUST be file_path = `filepath`.",
            backstory="You are fluent in Korean. You are a bookworm. Read and grasp everything in the document delicately",
            allow_delegation=False,
            verbose=True,
            llm="gpt-4o",
            tools=[
                FileReadTool(),
            ],
            max_iter=3,
            max_execution_time=1,
        )

    def fileSelector(self):
        return Agent(
            role="fileSelector",
            goal="Find out the path of all other files that correspond to the document and print them out.",
            backstory="You are a file search expert and fluent in Korean. You have a great ability to read and analyze the details of the file.",
            llm="gpt-4o-mini",
            allow_delegation=False,
            verbose=True,
        )

In [157]:
from crewai import Task


class Tasks:
    def markdownPathSearch(self, agent):
        return Task(
            description="Finds ALL the markdown files and inside {file_path} path",
            expected_output="Your final answer MUST be markdown file path. NEVER arbitrarily modify the path. Just Answer path in file_path",
            agent=agent,
            output_json=filePath,
            output_file="MarkdownPath.md",
            
        )

    def imgPathSearch(self, agent):
        return Task(
            description="Finds ALL the image files and inside {img_path} path. but NOT Include svg Image.",
            expected_output="Your final answer MUST be image path. svg images should NEVER be included. The file path symbol must be '/'. NEVER arbitrarily modify the path",
            agent=agent,
            output_json=filePath,
            output_file="ImgPath.md",
        )

    def mainFileSearch(self, agent, context):
        return Task(
            description="""
            Based on the markdownPathSearch, markdownPathSearch is in json format. Document Paths are stored in 'filePath' key value and is in List. Search Only one file path that can solve {question}.
            The path of the image NEVER ends with \\ and /.
            If the end of the path is \\ or /, remove this and explore file path.
            NEVER modify the file path in fileSelect.
            
            Read the entire contents of the file based on the file path and print it out.
            DON'T do this more than once
            """,
            expected_output="""
            Print out the entire contents of the file NEVER MODIFY.
            """,
            agent=agent,
            context=context,
            output_file="mainFileSearch.md",
        )

    def fileSelect(self, agent, context):
        return Task(
            description="""
            Based on the mainFileSearch, 
            There are other documents linked by the symbol '[[...]]' and '![[...]]' in that file NOT '[...]
            '[[...]]' symbol means a markdown file and '![[...]]' means an image file.
            
            Find all of the '[[...]]' and '![[...]]' and print out the ONLY file path associated with the word in it in markdownPathSearch or imgPathSearch. 
            All file paths should EXIST in that markdownPathSearch Output or imgPathSearch Output. 
            DON'T make it up and look for it.
            If the relevant document/image does not exist, JUST Return EMPTY List.",
            """,
            expected_output="""
            Your final answer MUST include the path of the first file and the path of other files within that file.
            It doesn't include ANYTHING other than file paths. 

            minaFile and relatedFiles Include ONLY markdown File!

            Example Answer 1
            {
                "mainFile": "./Algorithm/Algorithm Content/Tree/MST(Minimum Spanning Tree).md",
                "relatedFiles": [
                    "./Algorithm/Algorithm Content/Graph Theory/DFS(Depth-First Search).md",
                    "./Algorithm/Algorithm Content/Graph Theory/BFS(Breadth-First Search).md",
                    "./Algorithm/Algorithm Content/Tree/Union Find.md",
                ],
                "imageFiles": ["./Algorithm/Reference/Tree Reference/MST Ref/MST Graph.png",]
            }

            Example Answer 2
            {
                "mainFile": "./Algorithm/Algorithm Content/Graph Theory/BFS(Breadth-First Search).md",
                "relatedFiles": [],
                "imageFiles": [
                    "./Algorithm/Reference/Graph Theory Reference/BASE TREE.png",
                    "./Algorithm/Reference/Graph Theory Reference/BFS Ref/BFS Queue.png",
                ]
            }

            Example Answer 3
            {
                "mainFile": "./c/k.md",
                "relatedFiles": [
                    "./c/g.md",
                    "./c/c.md",
                    "./c/d.md",
                ],
                "imageFiles": []
            }
            """,
            agent=agent,
            context=context,
            output_json=associateFilePath,
            output_file="associateFilePath.md",
        )

In [158]:
from crewai import Crew

agent = Agents()
tasks = Tasks()

In [159]:
markdownPathSearcher = agent.markdownPathSearcher()

markdownPathSearcher_task = tasks.markdownPathSearch(markdownPathSearcher)

filePathCrew = Crew(
    agents=[markdownPathSearcher],
    tasks=[markdownPathSearcher_task],
    verbose=True,
)

filePathResult = filePathCrew.kickoff(
    dict(
        file_path=".\Algorithm\Algorithm Content",
    )
)

<>:13: SyntaxWarning: invalid escape sequence '\A'
<>:13: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Shind\AppData\Local\Temp\ipykernel_18528\4289515292.py:13: SyntaxWarning: invalid escape sequence '\A'
  file_path=".\Algorithm\Algorithm Content",
2025-01-12 02:49:34,658 - 23124 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: pathSearcher
## Task: Finds ALL the markdown files and inside .\Algorithm\Algorithm Content path


# Agent: pathSearcher
## Thought: I need to find all the markdown files in the specified directory .\Algorithm\Algorithm Content. To do this, I will list the files in that directory.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \".\\\\Algorithm\\\\Algorithm Content\"}"
## Tool Output: 
File paths: 
-.\Algorithm\Algorithm Content/Array\Binary Search.md
- .\Algorithm\Algorithm Content/Array\MITM(Meet in the middle).md
- .\Algorithm\Algorithm Content/Array\PBS(Parallel Binary Search).md
- .\Algorithm\Algorithm Content/Graph Theory\Articulation Points And Bridges.md
- .\Algorithm\Algorithm Content/Graph Theory\BFS(Breadth-First Search).md
- .\Algorithm\Algorithm Content/Graph Theory\CCW(Counter Clock Wise).md
- .\Algorithm\Algorithm Content/Graph Theory\DFS(Depth-First Search).md
- .\Algorithm\Algorithm Content/Graph Theory\Dijkstra's Algorithm.md
- .\Algor

In [72]:
imgPathSearcher = agent.imgPathSearcher()

imgPathSearcher_task = tasks.imgPathSearch(imgPathSearcher)

imgPathCrew = Crew(
    agents=[imgPathSearcher],
    tasks=[imgPathSearcher_task],
    verbose=True,
)

imgPathResult = imgPathCrew.kickoff(
    dict(
        img_path=".\Algorithm\Reference",
    )
)

<>:13: SyntaxWarning: invalid escape sequence '\A'
<>:13: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Shind\AppData\Local\Temp\ipykernel_18528\2886518380.py:13: SyntaxWarning: invalid escape sequence '\A'
  img_path=".\Algorithm\Reference",
2025-01-11 20:41:59,406 - 23124 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: pathSearcher
## Task: Finds ALL the image files and inside .\Algorithm\Reference path. but NOT Include svg Image.


# Agent: pathSearcher
## Thought: I need to find all image files in the `.\\Algorithm\\Reference` directory, excluding SVG images. I will start by listing the files in that directory.
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \".\\\\Algorithm\\\\Reference\"}"
## Tool Output: 
File paths: 
-.\Algorithm\Reference/BAEKJOON.png
- .\Algorithm\Reference/Array Reference\Binary Search Ref\Binary Search Flowchart.md
- .\Algorithm\Reference/Array Reference\Binary Search Ref\Binary Search Flowchart.png
- .\Algorithm\Reference/Array Reference\Binary Search Ref\Binary Search Flowchart.svg
- .\Algorithm\Reference/Array Reference\MITM Recursion Ref\MITM Recursion.md
- .\Algorithm\Reference/Array Reference\MITM Recursion Ref\MITM Recursion.png
- .\Algorithm\Reference/Array Reference\MITM Recursion Ref\MITM Recursion.svg
- .\Algorithm\Reference/Graph

In [73]:
mainFileSearcher = agent.mainFilesearcher()
fileSelector = agent.fileSelector()

mainFileSearcher_task = tasks.mainFileSearch(
    mainFileSearcher, [markdownPathSearcher_task]
)
fileSelector_task = tasks.fileSelect(
    fileSelector,
    [mainFileSearcher_task, markdownPathSearcher_task, imgPathSearcher_task],
)

fileSelectorCrew = Crew(
    agents=[
        mainFileSearcher,
        fileSelector,
    ],
    tasks=[
        mainFileSearcher_task,
        fileSelector_task,
    ],
    verbose=True,
)

fileSelectorResult = fileSelectorCrew.kickoff(
    dict(
        question="Finweck Tree가 뭐야?",
    )
)

2025-01-11 20:42:26,650 - 23124 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: mainFilesearcher
## Task: 
            Based on the markdownPathSearch, markdownPathSearch is in json format. Document Paths are stored in 'filePath' key value and is in List. Search Only one file path that can solve Finweck Tree가 뭐야?.
            The path of the image NEVER ends with \ and /.
            If the end of the path is \ or /, remove this and explore file path.
            NEVER modify the file path in fileSelect.
            
            Read the entire contents of the file based on the file path and print it out.
            DON'T do this more than once
            


# Agent: mainFilesearcher
## Thought: The question is about understanding what a Finweck Tree is, which is another name for a Fenwick Tree. Therefore, the most relevant document is the one titled "Fenwick Tree". 
Looking at the list of file paths in the context, I can see a file titled ".\Algorithm\Algorithm Content/Tree/Fenwick Tree.md". This file likely contains the information needed to understan

In [74]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

In [138]:
from langchain_unstructured import UnstructuredLoader
from langchain.document_loaders import UnstructuredImageLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import base64

def img_split(img_path):
    docs = []
    with open(img_path, "rb") as img_file:
        image_64 = base64.b64encode(img_file.read()).decode('utf-8')
        splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=500,
            chunk_overlap=60,
        )
        image_64_splitted = splitter.split_text(image_64)
        for img in image_64_splitted:
            docs.append(f"data:image/jpeg;base64,{img}")
    return docs


def document_split(file_path, includeCode=True):
    loader = UnstructuredLoader(file_path=file_path)
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500,
        chunk_overlap=60,
    )
    docs = loader.load()

    content = ""
    codeDocs = []
    codeDoc = ""
    flag = False
    for doc in docs:
        if "```" in doc.page_content:
            if flag:
                codeDocs.append(codeDoc)
                doc.page_content.replace("```", " ")
                flag = False
            else:
                codeDoc = ""
                flag = True

        if flag:
            codeDoc += doc.page_content + " "
        else:
            content += doc.page_content + " "
    textDocs = splitter.split_text(content)
    if includeCode:
        textDocs.extend(codeDocs)
    return textDocs

In [114]:
import json
import os.path

fileSelector_Json = json.loads(fileSelectorResult.raw)

relatedDocs = []
mainDocs = []

mainFilePath = fileSelector_Json["mainFile"]

if os.path.isfile(mainFilePath):
    mainDocs = document_split(mainFilePath)

for filePath in fileSelector_Json["relatedFiles"]:
    if os.path.isfile(filePath):
        relatedDocs.extend(document_split(filePath, False))

2025-01-11 21:02:12,177 - 23124 - filetype.py-filetype:412 - WARNING: libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
2025-01-11 21:02:12,531 - 23124 - filetype.py-filetype:412 - WARNING: libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
2025-01-11 21:02:12,869 - 23124 - filetype.py-filetype:412 - WARNING: libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
2025-01-11 21:02:13,209 - 23124 - filetype.py-filetype:412 - WARNING: libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [115]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

first_prompt = ChatPromptTemplate.from_template(
    """
    Your job is to find the right answer to the {question}.
    You are very good at using Korean and English.
    We have provied an existing answer to a certain point : {existing_content}
    We have the opportunity to refine the existing answer (only if needed) with some more context below.
    ------
    {context}
    ------
    Given the new context, refine the original answer.
    If the context ins't useful, RETURN the original answer.    
    """
)

first_chain = first_prompt | llm | StrOutputParser()

answer = ""
question = "BFS의 원리에 대해 알려줘"

for doc in mainDocs:
    answer = first_chain.invoke({"question" : question, "existing_content" : answer, "context" : doc})

In [78]:
refine_prompt  = ChatPromptTemplate.from_template(
    """
    Your job is to add supplementary content to understand the contents of the existing answer.
    You are very good at using Korean and English.
    We have provied an existing answer to a certain point : {existing_content}
    We have the opportunity to refine the existing answer (only if needed) with some more context below.
    ------
    {context}
    ------
    Given the new context, refine the original answer.
    If the context ins't useful, RETURN the original answer.    
    """
)

refine_chain = refine_prompt | llm | StrOutputParser()

for doc in relatedDocs:
    answer = refine_chain.invoke({"existing_content" : answer, "context" : doc})

answer

In [ ]:
img_path = "./Algorithm/Reference/Tree Reference/Fenwick Tree Ref/Fenwick Tree Struct Graph."

imgDocs = img_split("./luwak-2157626_1280.jpg")

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [140]:
img_refine_prompt = ChatPromptTemplate.from_template(
    """
    Your job is to express the contents of the image in one writing.
    You are very good at using Korean and English.
    We have provied an existing_content to a certain point : {existing_content}
    We have the opportunity to refine the existing answer (only if needed) with some more image url.
    ------
    {image_url}
    ------
    Given the image data, refine the original answer.
    If the context ins't useful, RETURN the original answer.    
    """
)

img_chain = img_refine_prompt | llm

img_content = ""

for img_doc in imgDocs:
    img_content = img_chain.invoke({"existing_content" : img_content, "image_url": img_path})

img_content

AIMessage(content="I'm sorry, but I cannot view images or access external content. However, if you provide me with a description or key details from the image, I can help you refine the existing content or create a new response based on that information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 396, 'total_tokens': 444, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-d501b207-c449-4ff6-b41a-d6460f6e3fd8-0', usage_metadata={'input_tokens': 396, 'output_tokens': 48, 'total_tokens': 444, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})